# **Welcome to the Product Recommendation System**

### Task 1 - Set up project environment

Installing the needed modules

In [1]:
!pip install openai==1.16.2 python-dotenv httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall: openai
    Found existing installation: openai 1.70.0
    Uninstalling openai-1.70.0:
      Successfully uninstalled openai-1.70.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.9.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


Importing the needed modules and setup the OpenAI API

In [33]:
import pandas as pd
import numpy as np
import os
import json
from openai import OpenAI
from dotenv import load_dotenv

from sklearn.metrics.pairwise import cosine_similarity

# Loading API key and organization ID from a dotenv file
load_dotenv(dotenv_path='/content/apikey.env.txt')

# Retrieving API key and organization ID from environment variables
APIKEY = os.getenv("APIKEY").replace("“","")
# ORGID = os.getenv("ORGID").replace("“","")

# Creating an instance of the OpenAI client with the provided API key and organization ID
client = OpenAI(
  # organization= ORGID,
  api_key=APIKEY
)

client

### Task 2 - Prepare the Kowledge Base

Loading the products dataset

In [3]:
products_data = pd.read_csv('/content/products_dataset.csv', encoding='utf-8')
products_data.head(10)

,product_id,title,description
0,P0,Men's 3X Large Carbon Heather Cotton/Polyester...,"This heavyweight, water-repellent hooded sweat..."
1,P1,Turmode 30 ft. RP TNC Female to RP TNC Male Ad...,If you need more length between your existing ...
2,P2,Large Tapestry Bolster Bed,Polyester cover resembling rich Italian tapest...
3,P3,16-Gauge-Sinks Vessel Sink in White with Faucet,It features a rectangle shape. This vessel set...
4,P4,Men's Crazy Horse 9'' Logger Boot - Steel Toe ...,This 9 in. black full grain leather logger boo...
5,P5,Mariana 6 ft. Multi-Color 3-Panel Screen Divider,With robust structure and sophisticated canvas...
6,P6,5 gal. #650C-2 Powdery Mist Semi-Gloss Interio...,BEHR PRO i300 Semi-Gloss Interior Paint has a ...
7,P7,7/8 in. x 4-1/2 in. x 0.045 in. Metal and Stai...,DEWALT High Performance 0.045 in. Metal Cuttin...
8,P8,Ring Gold Bar Cart,This Ring Bar Cart is sure to make a statement...
9,P9,Traditional Silver Wooden Vanity Table,This transitional vanity table proudly display...


Combining product `title` and `description`

In [4]:
products_data['keywords'] = products_data['title'] + ' ' + products_data['description']
products_data.head(10)

,product_id,title,description,keywords
0,P0,Men's 3X Large Carbon Heather Cotton/Polyester...,"This heavyweight, water-repellent hooded sweat...",Men's 3X Large Carbon Heather Cotton/Polyester...
1,P1,Turmode 30 ft. RP TNC Female to RP TNC Male Ad...,If you need more length between your existing ...,Turmode 30 ft. RP TNC Female to RP TNC Male Ad...
2,P2,Large Tapestry Bolster Bed,Polyester cover resembling rich Italian tapest...,Large Tapestry Bolster Bed Polyester cover res...
3,P3,16-Gauge-Sinks Vessel Sink in White with Faucet,It features a rectangle shape. This vessel set...,16-Gauge-Sinks Vessel Sink in White with Fauce...
4,P4,Men's Crazy Horse 9'' Logger Boot - Steel Toe ...,This 9 in. black full grain leather logger boo...,Men's Crazy Horse 9'' Logger Boot - Steel Toe ...
5,P5,Mariana 6 ft. Multi-Color 3-Panel Screen Divider,With robust structure and sophisticated canvas...,Mariana 6 ft. Multi-Color 3-Panel Screen Divid...
6,P6,5 gal. #650C-2 Powdery Mist Semi-Gloss Interio...,BEHR PRO i300 Semi-Gloss Interior Paint has a ...,5 gal. #650C-2 Powdery Mist Semi-Gloss Interio...
7,P7,7/8 in. x 4-1/2 in. x 0.045 in. Metal and Stai...,DEWALT High Performance 0.045 in. Metal Cuttin...,7/8 in. x 4-1/2 in. x 0.045 in. Metal and Stai...
8,P8,Ring Gold Bar Cart,This Ring Bar Cart is sure to make a statement...,Ring Gold Bar Cart This Ring Bar Cart is sure ...
9,P9,Traditional Silver Wooden Vanity Table,This transitional vanity table proudly display...,Traditional Silver Wooden Vanity Table This tr...


Creating the text embedding vectors

In [7]:
embeddings_response = client.embeddings.create(
  model="text-embedding-3-small",
  input=products_data['keywords'].to_list(),
  dimensions=512
)
vector_embeddings = [d.embedding for d in embeddings_response.data]
products_data['vectors'] = vector_embeddings
products_data.head(10)

,product_id,title,description,keywords,vectors
0,P0,Men's 3X Large Carbon Heather Cotton/Polyester...,"This heavyweight, water-repellent hooded sweat...",Men's 3X Large Carbon Heather Cotton/Polyester...,"[0.042664770036935806, 0.02092622220516205, -0..."
1,P1,Turmode 30 ft. RP TNC Female to RP TNC Male Ad...,If you need more length between your existing ...,Turmode 30 ft. RP TNC Female to RP TNC Male Ad...,"[0.04413316026329994, 0.009090634062886238, 0...."
2,P2,Large Tapestry Bolster Bed,Polyester cover resembling rich Italian tapest...,Large Tapestry Bolster Bed Polyester cover res...,"[0.042361605912446976, -0.06515178084373474, 0..."
3,P3,16-Gauge-Sinks Vessel Sink in White with Faucet,It features a rectangle shape. This vessel set...,16-Gauge-Sinks Vessel Sink in White with Fauce...,"[-0.049733716994524, -0.011679209768772125, 0...."
4,P4,Men's Crazy Horse 9'' Logger Boot - Steel Toe ...,This 9 in. black full grain leather logger boo...,Men's Crazy Horse 9'' Logger Boot - Steel Toe ...,"[0.026085881516337395, 0.048493191599845886, -..."
5,P5,Mariana 6 ft. Multi-Color 3-Panel Screen Divider,With robust structure and sophisticated canvas...,Mariana 6 ft. Multi-Color 3-Panel Screen Divid...,"[0.06058844178915024, 0.04209226742386818, 0.0..."
6,P6,5 gal. #650C-2 Powdery Mist Semi-Gloss Interio...,BEHR PRO i300 Semi-Gloss Interior Paint has a ...,5 gal. #650C-2 Powdery Mist Semi-Gloss Interio...,"[0.007497736252844334, -0.02833171933889389, 0..."
7,P7,7/8 in. x 4-1/2 in. x 0.045 in. Metal and Stai...,DEWALT High Performance 0.045 in. Metal Cuttin...,7/8 in. x 4-1/2 in. x 0.045 in. Metal and Stai...,"[-0.02177966572344303, -0.03028504177927971, -..."
8,P8,Ring Gold Bar Cart,This Ring Bar Cart is sure to make a statement...,Ring Gold Bar Cart This Ring Bar Cart is sure ...,"[-0.0004641585692297667, 0.06495653092861176, ..."
9,P9,Traditional Silver Wooden Vanity Table,This transitional vanity table proudly display...,Traditional Silver Wooden Vanity Table This tr...,"[0.03908809646964073, 1.859868280007504e-05, 0..."


### Task 3 - Retrieve relevant items based on user prompts

In [8]:
user_prompt = "I'm remodeling my bathroom and need a stylish white sink with a faucet included."

Let's find the embedding vector of the user prompt.

In [9]:
user_prompt_response = client.embeddings.create(
  model="text-embedding-3-small",
  input=[user_prompt],
  dimensions=512
)

user_prompt_embedding_vectors = [d.embedding for d in user_prompt_response.data]
print(user_prompt_embedding_vectors[:10])

[[-0.041445232927799225, -0.025848649442195892, -0.049512431025505066, -0.052806537598371506, -0.04406707361340523, -0.03352929651737213, -0.005697458982467651, 0.06117625907063484, -0.029949475079774857, -0.057949379086494446, -0.007605015765875578, 0.018890690058469772, -0.039092302322387695, -0.02833603508770466, 0.02875620312988758, 0.08739465475082397, 0.00044196273665875196, -0.07341150939464569, -0.028319228440523148, 0.08013417571783066, -0.031025102362036705, 0.02694108337163925, -0.03344526141881943, -0.06584850698709488, 0.026218395680189133, 0.016386497765779495, -0.04500824585556984, -0.006180650554597378, -0.026016715914011, 0.001717431005090475, -0.031747788190841675, -0.03818473964929581, -0.0443359799683094, 0.006088213995099068, -0.008176442235708237, 0.045478831976652145, 0.003621836192905903, -0.02510915696620941, 0.04194943234324455, -0.022638576105237007, -0.037949446588754654, 0.08241987973451614, 0.0406385138630867, -0.0007972661405801773, 0.04117632657289505, 0

Find similar products to the user prompt

In [18]:
product_vectors = [np.array(vec) for vec in products_data.vectors.values]
user_prompt_vectors = np.array(user_prompt_embedding_vectors)

similarity_matrix = cosine_similarity(product_vectors, user_prompt_vectors)
similarity_values = [v[0] for v in similarity_matrix]

products_data['similarity'] = similarity_values
products_data.sort_values(by = 'similarity', ascending = False, inplace = True)

products_data.head(10)

,product_id,title,description,keywords,vectors,similarity
946,P946,Cantrio Oval Drop-In Bathroom Sink in White,This Vitreous China round bathroom sink by Can...,Cantrio Oval Drop-In Bathroom Sink in White Th...,"[-0.030210549011826515, -0.012967006303369999,...",0.581895
3,P3,16-Gauge-Sinks Vessel Sink in White with Faucet,It features a rectangle shape. This vessel set...,16-Gauge-Sinks Vessel Sink in White with Fauce...,"[-0.049733716994524, -0.011679209768772125, 0....",0.580628
1665,P1665,Single Hole Single-Handle Bathroom Faucet in O...,"Charming and simplistic,This Waterfall Bathroo...",Single Hole Single-Handle Bathroom Faucet in O...,"[-0.038212429732084274, 0.03827337548136711, -...",0.579781
983,P983,Decorum Vitreous China Wall-Hung Rectangle Ves...,The Decorum Wall-Hung Bathroom Sink from Ameri...,Decorum Vitreous China Wall-Hung Rectangle Ves...,"[-0.0016402738401666284, -0.01754680462181568,...",0.563044
1638,P1638,Glass Vessel Sink in Smoky Black with Waterfal...,This flower petal formed vessel features grace...,Glass Vessel Sink in Smoky Black with Waterfal...,"[0.010049494914710522, 0.037063632160425186, 0...",0.562853
79,P79,24.38 in. x 14 in. x 6 in. White Above Counter...,Bathroom vessel sinks make the bathroom look m...,24.38 in. x 14 in. x 6 in. White Above Counter...,"[0.0010720123536884785, 0.02149433083832264, 0...",0.559397
347,P347,Modern in White Ceramic Rectangular Vessel Sin...,It features a rectangle shape. This vessel set...,Modern in White Ceramic Rectangular Vessel Sin...,"[-0.031610678881406784, -0.04411445930600166, ...",0.558828
780,P780,25.6 in. W x 15.7 in. D x 11.8 in. H Vanity in...,If you are looking for a high-end basin design...,25.6 in. W x 15.7 in. D x 11.8 in. H Vanity in...,"[-0.013337920419871807, 0.021912887692451477, ...",0.555517
1423,P1423,Miranda 59.5 in. W Double Bath Vanity in White...,Sleek and chic are the best words to describe ...,Miranda 59.5 in. W Double Bath Vanity in White...,"[-0.03355434164404869, 0.030692048370838165, 0...",0.553641
1352,P1352,31 in. W x 22 in. D Engineered Marble Vanity T...,Transform your bathroom with a beautiful engin...,31 in. W x 22 in. D Engineered Marble Vanity T...,"[-0.052632369101047516, 0.01649043709039688, 0...",0.548328


### Task 4 - Craft the context-rich prompt

Getting the top-10 similiar items

In [21]:
top_to_items = products_data.head(10)
top_to_items

,product_id,title,description,keywords,vectors,similarity
946,P946,Cantrio Oval Drop-In Bathroom Sink in White,This Vitreous China round bathroom sink by Can...,Cantrio Oval Drop-In Bathroom Sink in White Th...,"[-0.030210549011826515, -0.012967006303369999,...",0.581895
3,P3,16-Gauge-Sinks Vessel Sink in White with Faucet,It features a rectangle shape. This vessel set...,16-Gauge-Sinks Vessel Sink in White with Fauce...,"[-0.049733716994524, -0.011679209768772125, 0....",0.580628
1665,P1665,Single Hole Single-Handle Bathroom Faucet in O...,"Charming and simplistic,This Waterfall Bathroo...",Single Hole Single-Handle Bathroom Faucet in O...,"[-0.038212429732084274, 0.03827337548136711, -...",0.579781
983,P983,Decorum Vitreous China Wall-Hung Rectangle Ves...,The Decorum Wall-Hung Bathroom Sink from Ameri...,Decorum Vitreous China Wall-Hung Rectangle Ves...,"[-0.0016402738401666284, -0.01754680462181568,...",0.563044
1638,P1638,Glass Vessel Sink in Smoky Black with Waterfal...,This flower petal formed vessel features grace...,Glass Vessel Sink in Smoky Black with Waterfal...,"[0.010049494914710522, 0.037063632160425186, 0...",0.562853
79,P79,24.38 in. x 14 in. x 6 in. White Above Counter...,Bathroom vessel sinks make the bathroom look m...,24.38 in. x 14 in. x 6 in. White Above Counter...,"[0.0010720123536884785, 0.02149433083832264, 0...",0.559397
347,P347,Modern in White Ceramic Rectangular Vessel Sin...,It features a rectangle shape. This vessel set...,Modern in White Ceramic Rectangular Vessel Sin...,"[-0.031610678881406784, -0.04411445930600166, ...",0.558828
780,P780,25.6 in. W x 15.7 in. D x 11.8 in. H Vanity in...,If you are looking for a high-end basin design...,25.6 in. W x 15.7 in. D x 11.8 in. H Vanity in...,"[-0.013337920419871807, 0.021912887692451477, ...",0.555517
1423,P1423,Miranda 59.5 in. W Double Bath Vanity in White...,Sleek and chic are the best words to describe ...,Miranda 59.5 in. W Double Bath Vanity in White...,"[-0.03355434164404869, 0.030692048370838165, 0...",0.553641
1352,P1352,31 in. W x 22 in. D Engineered Marble Vanity T...,Transform your bathroom with a beautiful engin...,31 in. W x 22 in. D Engineered Marble Vanity T...,"[-0.052632369101047516, 0.01649043709039688, 0...",0.548328


Let's combine the information about all the related products.

In [24]:
combined_text = "\n\n".join("Product ID: "+top_10_items['product_id']+" Keywords: "+top_10_items['keywords'])
combined_text

"Product ID: P946 Keywords: Cantrio Oval Drop-In Bathroom Sink in White This Vitreous China round bathroom sink by Cantrio is the epitome of modern style. This refined statement piece is ideal for a faucet mount of your choosing and is equipped with a pre-drilled center drain. The stain resistant vitreous surface and unparalleled design make this drop-in sink both practical and sophisticated.\n\nProduct ID: P3 Keywords: 16-Gauge-Sinks Vessel Sink in White with Faucet It features a rectangle shape. This vessel set is designed to be installed as a undermount vessel set. It is constructed with ceramic. This vessel set comes with an enamel glaze finish in White color. It is designed for a deck mount faucet.\n\nProduct ID: P1665 Keywords: Single Hole Single-Handle Bathroom Faucet in Oil Rubbed Bronze Charming and simplistic,This Waterfall Bathroom Sink Faucet is a brilliant accessory for your bathroom.It always uses the best material to bring the best guarantee and quality to our customers.

Now let's craft our prompt

In [31]:
prompt = f"""
I want you to act as a customer service agent. We have the following products containing Product ID and keywords:
{combined_text}

i am using you for my ai app so just send json response, do not explain anything.
return the data in form that i could use JSON.parse(your_response) don't include any json keyword or anything

based on the user prompt and the products listed above, please provide 3 products as recommendations. Display the product ID
and keywords in json form of 3 recommended products
here is the user prompt: {user_prompt}
"""

prompt

"\nI want you to act as a customer service agent. We have the following products containing Product ID and keywords:\nProduct ID: P946 Keywords: Cantrio Oval Drop-In Bathroom Sink in White This Vitreous China round bathroom sink by Cantrio is the epitome of modern style. This refined statement piece is ideal for a faucet mount of your choosing and is equipped with a pre-drilled center drain. The stain resistant vitreous surface and unparalleled design make this drop-in sink both practical and sophisticated.\n\nProduct ID: P3 Keywords: 16-Gauge-Sinks Vessel Sink in White with Faucet It features a rectangle shape. This vessel set is designed to be installed as a undermount vessel set. It is constructed with ceramic. This vessel set comes with an enamel glaze finish in White color. It is designed for a deck mount faucet.\n\nProduct ID: P1665 Keywords: Single Hole Single-Handle Bathroom Faucet in Oil Rubbed Bronze Charming and simplistic,This Waterfall Bathroom Sink Faucet is a brilliant a

### Task 5 - Prompt the LLM to generate product recommendation

In [36]:
ai_response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    max_tokens=1200,
)

recommendations = ai_response.choices[0].message.content
recommended_products = json.loads(recommendations)
print("User Prompt: " + user_prompt)
print("Recommended Products: ")
print(recommended_products)

User Prompt: I'm remodeling my bathroom and need a stylish white sink with a faucet included.
Recommended Products: 
[{'Product ID': 'P3', 'Keywords': '16-Gauge-Sinks Vessel Sink in White with Faucet'}, {'Product ID': 'P347', 'Keywords': 'Modern in White Ceramic Rectangular Vessel Sink with Faucet Included with Overflow Drain Included'}, {'Product ID': 'P1638', 'Keywords': 'Glass Vessel Sink in Smoky Black with Waterfall Faucet and Pop-Up Drain in Chrome'}]


In [35]:
products_data.to_csv('products_dataset_with_vectors.csv', index=False)